**Text Generator**

In [ ]:


import os
import pptx
import random
import csv

directory_path = "/Users/garginandanpawar/Documents/iLovePDF_Output/ilovepdf-pdf-to-powerpoint"
output_directory_path = "/Users/garginandanpawar/Documents/iLovePDF_Output/Output"

min_length = 10  # minimum length of the paragraph in words
num_new_ppts = 400  # specify the number of new PowerPoints to generate

# Create a new CSV file to store information about the generated PowerPoints
csv_file_path = os.path.join(output_directory_path, "generated_powerpoints.csv")
csv_file = open(csv_file_path, 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['original_powerpoint', 'generated_name', 'original_slide_no', 'generated_slide_no'])

# ...
for i in range(num_new_ppts):
    # Create a new PowerPoint presentation
    new_prs = pptx.Presentation()

    # Randomly select three PowerPoint files
    selected_files = random.sample(os.listdir(directory_path), 3)
    original_powerpoints = []

    for index, filename in enumerate(selected_files):
        if filename.endswith(".pptx") or filename.endswith(".ppt"):
            pptx_file_path = os.path.join(directory_path, filename)
            prs = pptx.Presentation(pptx_file_path)

            # Select a random paragraph from valid paragraphs and add it as a new slide in the new PowerPoint presentation
            valid_paragraphs = []
            for slide_no, slide in enumerate(prs.slides):
                for shape in slide.shapes:
                    if hasattr(shape, "text"):
                        text = shape.text.strip()
                        paragraphs = text.split("\n\n")
                        valid_paragraphs += [(p, slide_no+1) for p in paragraphs if len(p.split()) >= min_length]
                if valid_paragraphs:
                    # Randomly select one valid paragraph
                    selected_paragraph, original_slide_no = random.choice(valid_paragraphs)
                    slide_layout = new_prs.slide_layouts[0]
                    slide = new_prs.slides.add_slide(slide_layout)
                    body_shape = slide.shapes.placeholders[1]
                    tf = body_shape.text_frame
                    tf.text = selected_paragraph
                    p = tf.paragraphs[0]
                    p.font.size = pptx.util.Pt(15)
                    generated_slide_no = len(new_prs.slides)
                    original_powerpoints.append((filename, original_slide_no, generated_slide_no))
                    break  # Only add one slide per presentation

            # If all three PowerPoint files have been processed, but the new PowerPoint presentation still has less than three slides
            # Add a new slide with a random paragraph from the current PowerPoint presentation
            if index == 2 and len(new_prs.slides) < 3:
                valid_paragraphs = []
                found_paragraph = False
                for slide_no, slide in enumerate(prs.slides):
                    if found_paragraph:
                        break
                    for shape in slide.shapes:
                        if hasattr(shape, "text"):
                            text = shape.text.strip()
                            paragraphs = text.split("\n\n")
                            valid_paragraphs = [(p, slide_no+1) for p in paragraphs if len(p.split()) >= min_length]
                            if valid_paragraphs:
                                # Randomly select one valid paragraph
                                selected_paragraph, original_slide_no = random.choice(valid_paragraphs)
                                slide_layout = new_prs.slide_layouts[0]
                                slide = new_prs.slides.add_slide(slide_layout)
                                body_shape = slide.shapes.placeholders[1]
                                tf = body_shape.text_frame
                                tf.text = selected_paragraph
                                p = tf.paragraphs[0]
                                p.font.size = pptx.util.Pt(15)
                                generated_slide_no = len(new_prs.slides)
                                original_powerpoints.append((filename, original_slide_no, generated_slide_no))
                                found_paragraph = True
                                break  # Only add one slide per presentation

    # Save the new PowerPoint presentation with a generated name
    generated_name = f"text_generated{i}.pptx"
    full_generated_path = os.path.join(output_directory_path, generated_name)
    new_prs.save(full_generated_path)

    # Write the information about the generated PowerPoint to the CSV file
    for ppt, original_slide_no, generated_slide_no in original_powerpoints:
        csv_writer.writerow([ppt, generated_name, original_slide_no, generated_slide_no])

# Close the CSV file
csv_file.close()

**Image Generator**

In [ ]:
#Works and generates new csv

import random
from pathlib import Path
from string import ascii_letters
from PIL import Image
import aspose.pydrawing as drawing
import aspose.slides as slides
from pptx import Presentation
import pandas as pd
import csv

# Number of new PowerPoint files to be created
number = 400
source_dir = Path("C:/Users/ASUS/Documents/M. Data science/thesis/UptoDate4.26/ilovepdf-pdf-to-powerpoint")
# Output directory where the new PowerPoint files will be saved
output_dir = Path("C:/Users/ASUS/Documents/M. Data science/thesis/UptoDate4.26/image_gen_data")
# Number of images to include in each new PowerPoint file
images_per_powerpoint = 3
# Whether to search for PowerPoint files recursively in the source directory
recursive = True

source_dir = Path(source_dir)
output_dir = Path(output_dir)
# Create the output directory if it doesn't exist
output_dir.mkdir(exist_ok=True)

# Create a temporary folder to store the extracted images
temp_folder = Path("temp")
temp_folder.mkdir(exist_ok=True)

# Create a new CSV file to store information about the generated PowerPoints
csv_file_path = output_dir / "generated_powerpoints.csv"
csv_file = open(csv_file_path, 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['original_powerpoint', 'generated_name', 'original_slide_no', 'generated_slide_no'])

# Function to get the image format
def get_image_format(image_type):
    # Map the image type string to the ImageFormat enumeration
    return {
        "jpeg": drawing.imaging.ImageFormat.jpeg,
        "emf": drawing.imaging.ImageFormat.emf,
        "bmp": drawing.imaging.ImageFormat.bmp,
        "png": drawing.imaging.ImageFormat.png,
        "wmf": drawing.imaging.ImageFormat.wmf,
        "gif": drawing.imaging.ImageFormat.gif,
    }.get(image_type, drawing.imaging.ImageFormat.jpeg)

# Function to extract images from the PowerPoint file
def extract_images(file: Path):
    image_paths = []
    original_slide_nos = []
    # List to store the path of the extracted images
    with slides.Presentation(str(file.absolute())) as presentation:
        # Loop through all the slides in the presentation
        for slide_no, slide in enumerate(presentation.slides):
            # Loop through all the shapes in the slide
            for shape_index, shape in enumerate(slide.shapes):
                shape_image = None

                # Check if the shape is a picture frame
                if type(shape) is slides.PictureFrame:
                    shape_image = shape.picture_format.picture.image
                # If the shape is an image, extract it
                if shape_image is not None:
                    # Get the content type of the image
                    image_type = shape_image.content_type.split("/")[1]
                    # Get the image format
                    image_format = get_image_format(image_type)
                    # Generate a random file name for the image
                    image_filename = "".join(random.choices(ascii_letters, k=10))
                    # Create the path for the image file
                    image_path = temp_folder / f"{image_filename}.{str(image_type).lower()}"

                    #Add the image path to the list
                    image_paths.append(image_path)
                    #Save the image to the file system
                    shape_image.system_image.save(
                        str(image_path.absolute()), image_format
                    )

                    #Resize the image if it's larger than 200x200
#                     with Image.open(str(image_path.absolute())) as img:
#                         if img.width > 200 or img.height > 200:
#                             img = img.resize((100, 100))
#                             img.save(str(image_path.absolute()))

                    # Record the slide number the image was extracted from
                    original_slide_nos.append(slide_no + 1)

        #Return the list of image paths and the list of original slide numbers
        return image_paths, original_slide_nos


#Get the list of PowerPoint files in the source directory
glob_regex = "**/*" if recursive else "*"
filenames = [f for f in source_dir.glob(glob_regex) if f.suffix in [".ppt", ".pptx"]]

for i in range(number):
    # Create a new PowerPoint presentation
    new_presentation = Presentation()
    # Generate a random file name for the new presentation
    presentation_name = f"image_generated{i}.pptx"

    # Loop through the number of images to be added to the new presentation
    for j in range(images_per_powerpoint):
        # Choose a random PowerPoint file from the list of source files
        source_file = random.choice(filenames)
        # Extract the images from the source file
        extracted_images, original_slide_nos = extract_images(source_file)
        
        # Check if at least one image is extracted
        if not extracted_images:
            continue
        
        # Choose a random image from the extracted images
        image_index = random.choice(range(len(extracted_images)))
        image_path = extracted_images[image_index]
        original_slide_no = original_slide_nos[image_index]

        # Insert the image into the new presentation
        slide = new_presentation.slides.add_slide(new_presentation.slide_layouts[6])
        pic = slide.shapes.add_picture(str(image_path.absolute()), 0, 0)

        # Write the information about the generated PowerPoint to the CSV file
        csv_writer.writerow([source_file.name, presentation_name, original_slide_no, len(new_presentation.slides)])

    # Save the new presentation to the output directory
    new_presentation.save(output_dir / presentation_name)


# Close the CSV file
csv_file.close()


**Image and Text Generator**

In [ ]:
import os #allworks
import csv
import random
from PIL import Image
from pathlib import Path
from pptx import Presentation
import aspose.slides as slides
from string import ascii_letters
import aspose.pydrawing as drawing

min_length = 10
num_new_ppts = 400

source_dir = Path(r"C:/Users/ASUS/Documents/M. Data science/thesis/UptoDate4.26/ilovepdf-pdf-to-powerpoint")
output_dir = Path(r"C:/Users/ASUS/Documents/M. Data science/thesis/UptoDate4.26/output_img_text")
recursive = False

csv_file = open(output_dir / 'generated_powerpoints.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['original_powerpoint', 'generated_name', 'original_slide_no', 'generated_slide_no'])

image_csv_file = open(output_dir / 'image_data.csv', 'w', newline='')
image_csv_writer = csv.writer(image_csv_file)
image_csv_writer.writerow(['original_powerpoint', 'generated_name', 'original_slide_no', 'generated_slide_no'])

source_dir = Path(source_dir)
output_dir = Path(output_dir)
output_dir.mkdir(exist_ok=True)

temp_folder = Path("temp")
temp_folder.mkdir(exist_ok=True)

def get_image_format(image_type):
    return {
        "jpeg": drawing.imaging.ImageFormat.jpeg,
        "emf": drawing.imaging.ImageFormat.emf,
        "bmp": drawing.imaging.ImageFormat.bmp,
        "png": drawing.imaging.ImageFormat.png,
        "wmf": drawing.imaging.ImageFormat.wmf,
        "gif": drawing.imaging.ImageFormat.gif,
    }.get(image_type, drawing.imaging.ImageFormat.jpeg)

def extract_images(file: str, ppt_name: str):
    image_paths = []
    with slides.Presentation(file) as presentation:
        for slide_no, slide in enumerate(presentation.slides):
            for shape_index, shape in enumerate(slide.shapes):
                shape_image = None
                if type(shape) is slides.PictureFrame:
                    shape_image = shape.picture_format.picture.image
                if shape_image is not None:
                    image_type = shape_image.content_type.split("/")[1]
                    image_format = get_image_format(image_type)
                    image_filename = "".join(random.choices(ascii_letters, k=10))
                    image_path = (
                            temp_folder / f"{image_filename}.{str(image_type).lower()}"
                    )
                    image_paths.append((image_path, slide_no+1, shape_index))
                    shape_image.system_image.save(
                        str(image_path), image_format
                    )
                    with Image.open(str(image_path)) as img:
                        if img.width > 200 or img.height > 200:
                            img = img.resize((100, 100))
                            img.save(str(image_path))
        return image_paths

image_info = []

for i in range(num_new_ppts):
    new_prs = Presentation()
    slide_counter = 0  # initialize a counter for the slides
    original_powerpoints = []

    while slide_counter < 3:  # ensure to generate exactly 3 slides
        filename = random.choice([file for file in os.listdir(source_dir) if file.endswith((".pptx", ".ppt"))])
        pptx_file_path = os.path.join(source_dir, filename)
        images = extract_images(pptx_file_path, filename)
        prs = Presentation(pptx_file_path)
        slide_height = new_prs.slide_height
        valid_paragraphs = []

        for slide_no, slide in enumerate(prs.slides):
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    text = shape.text.strip()
                    paragraphs = text.split("\n\n")
                    valid_paragraphs += [(p, slide_no+1) for p in paragraphs if len(p.split()) >= min_length]

        if valid_paragraphs and images:
            selected_paragraph, original_slide_no = random.choice(valid_paragraphs)
            slide_layout = new_prs.slide_layouts[0]
            slide = new_prs.slides.add_slide(slide_layout)

            body_shape = slide.shapes.placeholders[1]
            tf = body_shape.text_frame
            tf.text = selected_paragraph

            image_path, slide_no, shape_index = random.choice(images)
            slide.shapes.add_picture(str(image_path), 0, 0, height=slide_height)

            generated_slide_no = len(new_prs.slides)
            original_powerpoints.append((filename, original_slide_no, generated_slide_no))
            
            image_info.append((filename, f"text_image_generated{i}.pptx", slide_no, generated_slide_no))
            slide_counter += 1  # increment the counter

    generated_name = f"text_image_generated{i}.pptx"
    new_prs.save(output_dir / generated_name)

    for ppt, original_slide_no, generated_slide_no in original_powerpoints:
        csv_writer.writerow([ppt, generated_name, original_slide_no, generated_slide_no])

for info in image_info:
    image_csv_writer.writerow(info)

csv_file.close()
image_csv_file.close()

for f in temp_folder.glob("*"):
    f.unlink()
temp_folder.rmdir()
